In [ ]:
!pip install requests
!pip install scikit-learn
!pip install fsspec huggingface_hub
!pip install datasets

In [2]:
import pandas as pd
import re
import unicodedata
import requests
import json
from collections import Counter
import os
from io import StringIO
from sklearn.model_selection import train_test_split

In [4]:
def download_dataset():
    """Baixa o dataset diretamente da API da Hugging Face"""
    df = pd.read_csv("hf://datasets/sai1908/Mental_Health_Condition_Classification/Mental Health Text Dataset for Emotion and Condition Classification.csv")
    return df

def remove_accents(text):
    """Remove acentos de caracteres"""
    if pd.isna(text):
        return text
    # Normaliza para NFD (decompõe caracteres acentuados)
    nfd = unicodedata.normalize('NFD', text)
    without_accents = ''.join(c for c in nfd if unicodedata.category(c) != 'Mn')
    return without_accents
    
def remove_duplicates(df):
    """Remove textos duplicados do dataset"""
    before = len(df)
    df = df.drop_duplicates(subset="text", keep="first").reset_index(drop=True)
    after = len(df)
    removed = before - after
    
    print(f"\nRemovidos {removed} textos duplicados. Restam {after} amostras.")
    return df

def remove_class(df_normal):
    """Remove classe Normal do dataframe"""
    if 'normal' in df_normal['status'].values:
        print("\nRemovendo classe 'normal'...")
        df_normal = df_normal[df_normal['status'] != 'normal']
        print(f"Após remoção da classe normal: {len(df_normal)} amostras")
    else:
        print("\nClasse 'normal' não encontrada no dataset.")
    return df_normal

def remove_emojis(text):
    """Remove emojis do texto"""
    if pd.isna(text):
        return text
    
    # Padrão regex para capturar emojis
    emoji_pattern = re.compile(
        "["
        "\U0001F600-\U0001F64F"  # emoticons
        "\U0001F300-\U0001F5FF"  # símbolos & pictogramas
        "\U0001F680-\U0001F6FF"  # transporte & símbolos de mapa
        "\U0001F1E0-\U0001F1FF"  # bandeiras (iOS)
        "\U00002702-\U000027B0"
        "\U000024C2-\U0001F251"
        "\U0001f926-\U0001f937"
        "\U00010000-\U0010ffff"
        "\u2640-\u2642"
        "\u2600-\u2B55"
        "\u200d"
        "\u23cf"
        "\u23e9"
        "\u231a"
        "\ufe0f"  # dingbats
        "\u3030"
        "]+", flags=re.UNICODE
    )
    
    return emoji_pattern.sub(r'', text)

def clean_text(text):
    """Limpa e padroniza o texto"""
    if pd.isna(text):
        return ""
    
    text = str(text)
    
    # Remover aspas duplas e simples no início e fim
    text = text.strip('"\'')
    
    # Converter para minúsculas
    text = text.lower()
    
    # Remover acentos
    text = remove_accents(text)
    
    # Manter apenas letras, números, espaços e pontuação específica
    # Mantém: apóstrofo, vírgula, ponto, exclamação, interrogação, espaços
    text = re.sub(r"[^a-zA-Z0-9\s',\.!?]", " ", text)
    
    text = re.sub(r'\s+', ' ', text)
    
    text = text.strip()
    
    return text

def analyze_character_distribution(df):
    """Analisa a distribuição de caracteres por classe"""
    
    char_stats = {}
    for class_name in df['status'].unique():
        class_df = df[df['status'] == class_name]
        class_df['char_count'] = class_df['text'].str.len()
        
        total_chars = class_df['char_count'].sum()
        avg_chars = class_df['char_count'].mean()
        median_chars = class_df['char_count'].median()
        min_chars = class_df['char_count'].min()
        max_chars = class_df['char_count'].max()
        
        char_stats[class_name] = {
            'samples': len(class_df),
            'total_chars': total_chars,
            'avg_chars': avg_chars,
            'median_chars': median_chars,
            'min_chars': min_chars,
            'max_chars': max_chars
        }
        
        print(f"\nClasse '{class_name}':")
        print(f"  Amostras: {len(class_df):,}")
        print(f"  Total de caracteres: {total_chars:,}")
        print(f"  Média de caracteres por texto: {avg_chars:.1f}")
        print(f"  Mediana de caracteres: {median_chars:.1f}")
        print(f"  Min-Max caracteres: {min_chars} - {max_chars}")
    
    return char_stats

def balance_by_characters(df, target_chars_per_class=None):
    """Balanceia as classes baseado na quantidade total de caracteres"""
    
    # Adiciona contagem de caracteres
    df['char_count'] = df['text'].str.len()
    
    # Calcula caracteres por classe
    char_per_class = {}
    for class_name in df['status'].unique():
        class_df = df[df['status'] == class_name]
        char_per_class[class_name] = class_df['char_count'].sum()
    
    # Define o target baseado na menor classe se não especificado
    if target_chars_per_class is None:
        target_chars_per_class = min(char_per_class.values())
    
    print(f"Balanceando para {target_chars_per_class:,} caracteres por classe...")
    
    balanced_dfs = []
    
    for class_name in df['status'].unique():
        class_df = df[df['status'] == class_name].copy()
        current_chars = char_per_class[class_name]
        
        print(f"\nProcessando classe '{class_name}':")
        print(f"  Caracteres atuais: {current_chars:,}")
        print(f"  Caracteres alvo: {target_chars_per_class:,}")
        
        if current_chars >= target_chars_per_class:
            # Se tem mais caracteres que necessário, seleciona amostras até atingir o target
            class_df = class_df.sort_values('char_count', ascending=False)  # Ordena por tamanho
            
            selected_indices = []
            accumulated_chars = 0
            
            for idx, row in class_df.iterrows():
                if accumulated_chars + row['char_count'] <= target_chars_per_class:
                    selected_indices.append(idx)
                    accumulated_chars += row['char_count']
                else:
                    # Se adicionar este texto ultrapassar o limite, para
                    break
            
            balanced_class = class_df.loc[selected_indices]
            
        else:
            # Se tem menos caracteres, replica amostras
            balanced_class = class_df.copy()
            current_chars_balanced = current_chars
            
            while current_chars_balanced < target_chars_per_class:
                # Calcula quantos caracteres ainda faltam
                remaining_chars = target_chars_per_class - current_chars_balanced
                
                # Seleciona amostras para replicar (começando pelas maiores)
                class_df_sorted = class_df.sort_values('char_count', ascending=False)
                
                for idx, row in class_df_sorted.iterrows():
                    if current_chars_balanced + row['char_count'] <= target_chars_per_class:
                        # Adiciona uma cópia desta amostra
                        new_row = row.copy()
                        balanced_class = pd.concat([balanced_class, new_row.to_frame().T], ignore_index=True)
                        current_chars_balanced += row['char_count']
                        
                        if current_chars_balanced >= target_chars_per_class:
                            break
                
                # Evita loop infinito
                if current_chars_balanced == balanced_class['char_count'].sum():
                    break
        
        final_chars = balanced_class['char_count'].sum()
        print(f"  Resultado: {len(balanced_class)} amostras, {final_chars:,} caracteres")
        
        balanced_dfs.append(balanced_class)
    
    return pd.concat(balanced_dfs, ignore_index=True)

def split_dataset(df, train_ratio=0.6, val_ratio=0.2, test_ratio=0.2, random_state=42):
    """Divide o dataset em treino, validação e teste mantendo a proporção das classes"""
    print(f"\n=== DIVISÃO DO DATASET ===")
    print(f"Treino: {train_ratio*100}%, Validação: {val_ratio*100}%, Teste: {test_ratio*100}%")
    
    # Primeiro divide em treino e temp (val+test)
    X = df['text']
    y = df['status']
    
    X_train, X_temp, y_train, y_temp = train_test_split(
        X, y, test_size=(val_ratio + test_ratio), 
        stratify=y, random_state=random_state
    )
    
    # Depois divide temp em validação e teste
    val_size = val_ratio / (val_ratio + test_ratio)
    X_val, X_test, y_val, y_test = train_test_split(
        X_temp, y_temp, test_size=(1-val_size), 
        stratify=y_temp, random_state=random_state
    )
    
    # Cria os DataFrames finais
    train_df = pd.DataFrame({'text': X_train, 'status': y_train})
    val_df = pd.DataFrame({'text': X_val, 'status': y_val})
    test_df = pd.DataFrame({'text': X_test, 'status': y_test})
    
    print(f"\nDivisão final:")
    print(f"  Treino: {len(train_df):,} amostras")
    print(f"  Validação: {len(val_df):,} amostras") 
    print(f"  Teste: {len(test_df):,} amostras")
    
    print(f"\nDistribuição por conjunto:")
    for dataset_name, dataset in [("Treino", train_df), ("Validação", val_df), ("Teste", test_df)]:
        print(f"\n{dataset_name}:")
        class_counts = dataset['status'].value_counts()
        for class_name, count in class_counts.items():
            percentage = (count / len(dataset)) * 100
            print(f"  {class_name}: {count} ({percentage:.1f}%)")
    
    return train_df, val_df, test_df

In [5]:
# Baixa o dataset
df_original = download_dataset()

print(f"\nDataset original: {len(df_original)} amostras")
print(f"Colunas: {list(df_original.columns)}")
print("\nDistribuição das classes original:")
print(df_original['status'].value_counts())

/home/rodrigo/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



Dataset original: 103488 amostras
Colunas: ['text', 'status']

Distribuição das classes original:
status
anxiety                 17620
normal                  16068
depression              15901
stress                  15230
personality disorder    13915
bipolar                 13708
suicidal                11046
Name: count, dtype: int64


In [6]:
# Limpar duplicados
df_original = remove_duplicates(df_original)

# Remove a classe 'normal' se existir
df = remove_class(df_original)

# Limpa os textos
df['text'] = df['text'].apply(clean_text)

# Remove linhas com texto vazio após limpeza
df = df[df['text'].str.len() > 0]

print(f"Após limpeza: {len(df)} amostras")
print("\nDistribuição das classes após limpeza:")
print(df['status'].value_counts())

# Encontra a menor classe para definir o tamanho do balanceamento
class_counts = df['status'].value_counts()
min_class_size = class_counts.min()

print(f"\nMenor classe tem {min_class_size} amostras")

train_df, val_df, test_df = split_dataset(df, 
                                          train_ratio=0.6, 
                                          val_ratio=0.2, 
                                          test_ratio=0.2)

print(f"Treino antes do balanceamento: {len(train_df)} amostras")
print("Distribuição do treino antes do balanceamento:")
print(train_df['status'].value_counts())

# Analisa distribuição de caracteres do treino antes do balanceamento
print("\n=== ANÁLISE DO TREINO ANTES DO BALANCEAMENTO ===")
analyze_character_distribution(train_df)

# Balanceia apenas o treino
train_df_balanced = balance_by_characters(train_df)

print(f"\nTreino após balanceamento: {len(train_df_balanced)} amostras")
print("Distribuição final do treino:")
print(train_df_balanced['status'].value_counts())

# Analisa distribuição após balanceamento do treino
print("\n=== TREINO APÓS BALANCEAMENTO ===")
analyze_character_distribution(train_df_balanced)


Removidos 24 textos duplicados. Restam 103464 amostras.

Removendo classe 'normal'...
Após remoção da classe normal: 87396 amostras


/tmp/ipykernel_55553/2897239175.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['text'].apply(clean_text)


Após limpeza: 87396 amostras

Distribuição das classes após limpeza:
status
anxiety                 17613
depression              15891
stress                  15227
personality disorder    13915
bipolar                 13708
suicidal                11042
Name: count, dtype: int64

Menor classe tem 11042 amostras

=== DIVISÃO DO DATASET ===
Treino: 60.0%, Validação: 20.0%, Teste: 20.0%

Divisão final:
  Treino: 52,437 amostras
  Validação: 17,479 amostras
  Teste: 17,480 amostras

Distribuição por conjunto:

Treino:
  anxiety: 10568 (20.2%)
  depression: 9534 (18.2%)
  stress: 9136 (17.4%)
  personality disorder: 8349 (15.9%)
  bipolar: 8225 (15.7%)
  suicidal: 6625 (12.6%)

Validação:
  anxiety: 3522 (20.1%)
  depression: 3178 (18.2%)
  stress: 3045 (17.4%)
  personality disorder: 2783 (15.9%)
  bipolar: 2742 (15.7%)
  suicidal: 2209 (12.6%)

Teste:
  anxiety: 3523 (20.2%)
  depression: 3179 (18.2%)
  stress: 3046 (17.4%)
  personality disorder: 2783 (15.9%)
  bipolar: 2741 (15.7%)
  

/tmp/ipykernel_55553/3227154787.py:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  class_df['char_count'] = class_df['text'].str.len()
/tmp/ipykernel_55553/3227154787.py:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  class_df['char_count'] = class_df['text'].str.len()
/tmp/ipykernel_55553/3227154787.py:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pand


Classe 'personality disorder':
  Amostras: 8,349
  Total de caracteres: 2,586,024
  Média de caracteres por texto: 309.7
  Mediana de caracteres: 178.0
  Min-Max caracteres: 39 - 27475

Classe 'stress':
  Amostras: 9,136
  Total de caracteres: 2,627,836
  Média de caracteres por texto: 287.6
  Mediana de caracteres: 183.0
  Min-Max caracteres: 34 - 8351

Classe 'depression':
  Amostras: 9,534
  Total de caracteres: 8,392,681
  Média de caracteres por texto: 880.3
  Mediana de caracteres: 586.0
  Min-Max caracteres: 3 - 19822

Classe 'suicidal':
  Amostras: 6,625
  Total de caracteres: 5,024,766
  Média de caracteres por texto: 758.5
  Mediana de caracteres: 460.0
  Min-Max caracteres: 10 - 27374

Classe 'bipolar':
  Amostras: 8,225
  Total de caracteres: 3,198,091
  Média de caracteres por texto: 388.8
  Mediana de caracteres: 230.0
  Min-Max caracteres: 29 - 26066

Classe 'anxiety':
  Amostras: 10,568
  Total de caracteres: 4,227,763
  Média de caracteres por texto: 400.1
  Mediana d

/tmp/ipykernel_55553/3227154787.py:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  class_df['char_count'] = class_df['text'].str.len()
/tmp/ipykernel_55553/3227154787.py:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  class_df['char_count'] = class_df['text'].str.len()
/tmp/ipykernel_55553/3227154787.py:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pand

{'personality disorder': {'samples': 8349,
  'total_chars': np.int64(2586024),
  'avg_chars': np.float64(309.7405677326626),
  'median_chars': np.float64(178.0),
  'min_chars': np.int64(39),
  'max_chars': np.int64(27475)},
 'stress': {'samples': 8798,
  'total_chars': np.int64(2585994),
  'avg_chars': np.float64(293.92975676290064),
  'median_chars': np.float64(185.0),
  'min_chars': np.int64(134),
  'max_chars': np.int64(8351)},
 'depression': {'samples': 758,
  'total_chars': np.int64(2585347),
  'avg_chars': np.float64(3410.7480211081793),
  'median_chars': np.float64(2902.5),
  'min_chars': np.int64(2139),
  'max_chars': np.int64(19822)},
 'suicidal': {'samples': 1116,
  'total_chars': np.int64(2584885),
  'avg_chars': np.float64(2316.2051971326164),
  'median_chars': np.float64(1915.5),
  'min_chars': np.int64(1269),
  'max_chars': np.int64(27374)},
 'bipolar': {'samples': 4539,
  'total_chars': np.int64(2585853),
  'avg_chars': np.float64(569.6966292134831),
  'median_chars': np

In [7]:
def save_file(df_file):
    """Salva os arquivos na pasta 'treinamento'"""
    # Cria a pasta 'treinamento' se não existir
    os.makedirs("treinamento", exist_ok=True)
    
    for class_name in df_file['status'].unique():
        class_texts = df_file[df_file['status'] == class_name]['text'].tolist()
        
        # Nome do arquivo dentro da pasta 'treinamento'
        filename = os.path.join("treinamento", f"{class_name}.txt")
        
        # Salva os textos, um por linha
        with open(filename, 'w', encoding='utf-8') as f:
            for text in class_texts:
                f.write(text + '\n')
        
        print(f"Salvo: {filename} ({len(class_texts)} textos)")

# Exemplo de uso
save_file(train_df_balanced)

Salvo: treinamento/personality disorder.txt (8349 textos)
Salvo: treinamento/stress.txt (8798 textos)
Salvo: treinamento/depression.txt (758 textos)
Salvo: treinamento/suicidal.txt (1116 textos)
Salvo: treinamento/bipolar.txt (4539 textos)
Salvo: treinamento/anxiety.txt (2758 textos)


In [8]:
# Cria as pastas se não existirem
os.makedirs("validacao", exist_ok=True)
os.makedirs("teste", exist_ok=True)

# Salvar validação dentro da pasta "validacao"
val_df.to_csv(os.path.join("validacao", "validacao.csv"), index=False)

# Salvar teste dentro da pasta "teste"
test_df.to_csv(os.path.join("teste", "teste.csv"), index=False)

print("Arquivos salvos em suas respectivas pastas.")

Arquivos salvos em suas respectivas pastas.
